In [21]:
import sounderpy as spy
from datetime import datetime as dt
import numpy as np
import pandas as pd
import warnings
from associate_obs import plot_radiosonde_data
from matplotlib import pyplot as plt

In [22]:
date_i_want = dt(2022, 9, 17, 0, 0, 0)

In [23]:
# Load ACARS profiles
acars_all_list = []
airports_i_want = {'IAH' : {'latitude' : 29.9844353, 'longitude' : -95.3414425},
                'HOU' : {'latitude' : 29.6457998, 'longitude' : -95.2772316},
                'LBX' : {'latitude' : 29.1086389, 'longitude' : -95.4620833},
                'GLS' : {'latitude' : 29.2653333, 'longitude' : -94.8604167},
                'EFD' : {'latitude' : 29.6073333, 'longitude' : -95.1587500},
                'IWS' : {'latitude' : 29.8181944, 'longitude' : -95.6726111},
                'SGR' : {'latitude' : 29.6222486, 'longitude' : -95.6565342},
                'DWH' : {'latitude' : 30.0617791, 'longitude' : -95.5527884},
                'CXO' : {'latitude' : 30.3533955, 'longitude' : -95.4150819},
                'HPY' : {'latitude' : 29.7860833, 'longitude' : -94.9526667}
                }
for hour_to_pull in range(24):
    acars_conn = spy.acars_data(str(date_i_want.year).zfill(4), str(date_i_want.month).zfill(2), str(date_i_want.day).zfill(2), str(hour_to_pull).zfill(2))
    try:
        this_hour_acars_list = [prof for prof in acars_conn.list_profiles() if prof[:3] in airports_i_want.keys()]
    except Exception as e:
        if 'HTTP Error' in str(e):
            continue
        else:
            raise e
    acars_all_list.extend(this_hour_acars_list)
acars_profile_dts = []
acars_profile_lons = []
acars_profile_lats = []
acars_profile_files = []
for acars_loctime in acars_all_list:
    acars_profile_dts.append(date_i_want.replace(hour=int(acars_loctime[-4:-2]), minute=int(acars_loctime[-2:])))
    acars_profile_lats.append(airports_i_want[acars_loctime[:3]]['latitude'])
    acars_profile_lons.append(airports_i_want[acars_loctime[:3]]['longitude'])
    acars_profile_files.append('ACARS+'+acars_loctime)

> LIST ACARS PROFILES FUNCTION
  ---------------------------------
> AVAILABLE ACARS PROFILES FOR 2022-09-17 00Z...
> COMPLETE --------
> RUNTIME: 00:00:00
> LIST ACARS PROFILES FUNCTION
  ---------------------------------
> AVAILABLE ACARS PROFILES FOR 2022-09-17 01Z...
> COMPLETE --------
> RUNTIME: 00:00:00
> LIST ACARS PROFILES FUNCTION
  ---------------------------------
> AVAILABLE ACARS PROFILES FOR 2022-09-17 02Z...
> COMPLETE --------
> RUNTIME: 00:00:00
> LIST ACARS PROFILES FUNCTION
  ---------------------------------
> AVAILABLE ACARS PROFILES FOR 2022-09-17 03Z...
> COMPLETE --------
> RUNTIME: 00:00:00
> LIST ACARS PROFILES FUNCTION
  ---------------------------------
> AVAILABLE ACARS PROFILES FOR 2022-09-17 04Z...
> COMPLETE --------
> RUNTIME: 00:00:00
> LIST ACARS PROFILES FUNCTION
  ---------------------------------
> AVAILABLE ACARS PROFILES FOR 2022-09-17 05Z...
> COMPLETE --------
> RUNTIME: 00:00:00
> LIST ACARS PROFILES FUNCTION
  -------------------------------

In [24]:
for f, this_dt, this_lon, this_lat in zip(acars_profile_files, acars_profile_dts, acars_profile_lons, acars_profile_lats):
    this_pydt = this_dt
    if f.startswith('ACARS'):
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore')
            acars_conn = spy.acars_data(str(this_pydt.year).zfill(4), str(this_pydt.month).zfill(2), str(this_pydt.day).zfill(2), str(this_pydt.hour).zfill(2))
            try:
                sounding = acars_conn.get_profile(f.replace('ACARS+', ''), hush=True, clean_it=True)
            except Exception as e:
                if 'HTTP Error' in str(e):
                    continue
                else:
                    raise e
        this_sonde_data = pd.DataFrame(
            {'pres' : sounding['p'].to('hPa').m,
            'tdry' : sounding['T'].to('degC').m,
            'dp' : sounding['Td'].to('degC').m,
            'u_wind' : sounding['u'].to('m/s').m,
            'v_wind' : sounding['v'].to('m/s').m,
            'alt' : sounding['z'].to('m').m}
        )
        this_fig = plot_radiosonde_data(this_sonde_data, this_pydt)
        siteid = f.replace('ACARS+', '').split('_')[0]
        this_fig.savefig(f'acars-figs/{siteid}_{this_pydt.strftime("%Y%m%d_%H%M")}.png')
        this_sonde_data.to_csv(f'acars-sondes/{siteid}_{this_pydt.strftime("%Y%m%d_%H%M")}.csv', index=False)

> ACARS DATA ACCESS FUNCTION
  ---------------------------------
    > COMPLETE --------
    > RUNTIME: 00:00:01
> ACARS DATA ACCESS FUNCTION
  ---------------------------------
    > COMPLETE --------
    > RUNTIME: 00:00:00
> ACARS DATA ACCESS FUNCTION
  ---------------------------------
    > COMPLETE --------
    > RUNTIME: 00:00:00
> ACARS DATA ACCESS FUNCTION
  ---------------------------------
    > COMPLETE --------
    > RUNTIME: 00:00:01
> ACARS DATA ACCESS FUNCTION
  ---------------------------------
    > COMPLETE --------
    > RUNTIME: 00:00:00
> ACARS DATA ACCESS FUNCTION
  ---------------------------------
    > COMPLETE --------
    > RUNTIME: 00:00:00
> ACARS DATA ACCESS FUNCTION
  ---------------------------------
    > COMPLETE --------
    > RUNTIME: 00:00:00
> ACARS DATA ACCESS FUNCTION
  ---------------------------------
    > COMPLETE --------
    > RUNTIME: 00:00:00
> ACARS DATA ACCESS FUNCTION
  ---------------------------------
    > COMPLETE --------
    > R